In [155]:
import pandas as pd
from statistics import median
import csv

In [156]:
project_list = [
    'gradle.csv',
    'okhttp.csv',
    'android.csv',
    'cloudify.csv',
    'graylog2-server.csv'
]

In [157]:
r_file_name = 'ci_skipped_dynamic_batching.csv'
result_file = open(r_file_name, 'w')
result_headers = ['project', 'algorithm', 'batch_size', 'builds_saved', 'median_delay', 'testall_size']
writer = csv.writer(result_file)
writer.writerow(result_headers)

69

In [158]:
batch_total = 0

In [159]:
def output_values(tr_results):
    l = []
    i = 0
    while i < len(tr_results):
        if tr_results[i] == 'passed':
            l.append(1)
        else:
            l.append(0)
        i += 1
    
    return l

In [160]:
def batchstop4(commits):
    global batch_total
    
    l = len(commits)//2
    
    batch_total += 1        
    
    if 0 in commits:
        
        if len(commits) == 4:
            batch_total += 4
            return commits[0] & commits[1] & commits[2] & commits[3]
        
        return batchbisect(commits[:l]) & batchbisect(commits[l:])
    
    return 1

In [161]:
def batchbisect(commits):
    global batch_total
    
    l = len(commits)//2
    
    batch_total += 1
    if len(commits) == 1:
        return commits[0]
    
    if 0 in commits:
        return batchbisect(commits[:l]) & batchbisect(commits[l:])
    
    return 1

In [162]:
def batchbisect_dynamic(commits):
    global batch_total
    
    batch_size = 16
    length = len(commits)
    builds = 0
    i = 0
    delays = []
    
    while i < length:
        batch = commits[i:i+batch_size]
        fails_per = 100*batch.count(0)/batch_size
        
        delays.extend([batch_size-x-1 for x in range(batch_size)])
        
        batch_total = 0
        res = batchbisect(batch)
        builds += batch_total
        print('Building {} {} {} {}'.format(batch_size, batch, batch_total, builds))
        
        i += batch_size
        
        if res == 0:
            
            if fails_per > 50 & batch[-1] == 0:
                #if batch has a lot of failures then definitely split
                
                if batch_size == 1:
                    continue
                else:
                    batch_size //= 2
            
            if fails_per < 20 & batch[-1] == 1:
                continue
        
        else:
            if batch_size == 16:
                continue
            else:
                batch_size *= 2
    
    return builds, delays
            

In [163]:
def batchstop4_dynamic(commits):
    global batch_total
    
    batch_size = 16
    length = len(commits)
    builds = 0
    i = 0
    delays = []
    
    while i < length:
        batch = commits[i:i+batch_size]
        fails_per = 100*batch.count(0)/batch_size
        delays.extend([batch_size-x-1 for x in range(batch_size)])
        
        batch_total = 0
        res = batchstop4(batch)
        builds += batch_total
        print('Building {} {} {} {}'.format(batch_size, batch, batch_total, builds))
        
        i += batch_size
        
        if res == 0:
            if fails_per > 50 & batch[-1] == 0:
                #if batch has a lot of failures then definitely split
                
                if batch_size == 4:
                    continue
                else:
                    batch_size //= 2
            
            if fails_per < 20 & batch[-1] == 1:
                continue
        
        else:
            if batch_size == 16:
                continue
            else:
                batch_size *= 2
    
    return builds, delays

In [164]:
for p in project_list:
    data = pd.read_csv('ci_skip_data/extracted_project_travis/' + p)
    total_length = len(data) - 100
    data = data[ data['verdict'] == 0]
    data = data.iloc[100:]
    results = output_values(data['tr_status'].tolist())
    builds, delays = batchbisect_dynamic(results)
    reqd = 100-(100*builds/total_length)
    fr = 100*results.count(0)/total_length
    
    writer.writerow([p, 'BATCHBISECT', '-1', reqd, median(delays), total_length])
    
    

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 1
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 2
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1] 11 13
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 14
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 15
Building 16 [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1] 9 24
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 25
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1] 9 34
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 35
Building 16 [1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1] 17 52
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 53
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0] 9 62
Building 8 [1, 1, 0, 1, 1, 1, 1, 1] 7 69
Building 4 [1, 1, 1, 1] 1 70
Building 8 [1, 1, 1, 1, 1, 1, 0, 1] 7 77
Building 4 [1, 1, 1, 1] 1 78
Building 8 [0, 1, 1, 1, 1, 1, 1, 1] 7 85
Building 4 [1, 1, 1, 1] 1 86
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 87
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 491
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 492
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 493
Building 16 [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 9 502
Building 8 [1, 1, 1, 1, 1, 1, 1, 0] 7 509
Building 4 [1, 0, 1, 1] 5 514
Building 2 [1, 1] 1 515
Building 4 [1, 1, 1, 1] 1 516
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 517
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1] 13 530
Building 8 [1, 1, 1, 0, 1, 1, 1, 1] 7 537
Building 4 [1, 1, 1, 1] 1 538
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 539
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 540
Building 16 [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1] 9 549
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 550
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0] 9 559
Building 8 [1, 0, 0, 0, 1, 1, 1, 1] 9 568
Building 4 [1, 1, 1, 1] 1 569
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 570
Building 16 [1, 1, 1, 1, 

In [165]:
for p in project_list:
    data = pd.read_csv('ci_skip_data/extracted_project_travis/' + p)
    total_length = len(data) - 100
    data = data[ data['verdict'] == 0]
    data = data.iloc[100:]
    results = output_values(data['tr_status'].tolist())
    builds, delays = batchstop4_dynamic(results)
    reqd = 100-(100*builds/total_length)
    fr = 100*results.count(0)/total_length
    
    writer.writerow([p, 'BATCHSTOP4', '-1', reqd, median(delays), total_length])
    
    #print('{:>40} {:>10} {:>30} {:>10}'.format(p, len(results), reqd, median(delays)))

Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 1
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 2
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1] 11 13
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 14
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1 15
Building 16 [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1] 9 24
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 25
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1] 9 34
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 35
Building 16 [1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1] 17 52
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 53
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0] 9 62
Building 8 [1, 1, 0, 1, 1, 1, 1, 1] 7 69
Building 4 [1, 1, 1, 1] 1 70
Building 8 [1, 1, 1, 1, 1, 1, 0, 1] 7 77
Building 4 [1, 1, 1, 1] 1 78
Building 8 [0, 1, 1, 1, 1, 1, 1, 1] 7 85
Building 4 [1, 1, 1, 1] 1 86
Building 8 [1, 1, 1, 1, 1, 1, 1, 1] 1 87
Building 16 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Building 16 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 31 31
Building 8 [0, 0, 0, 0, 0, 0, 0, 0] 15 46
Building 4 [0, 0, 0, 0] 5 51
Building 4 [0, 0, 0, 0] 5 56
Building 4 [0, 0, 0, 0] 5 61
Building 4 [0, 0, 1, 0] 5 66
Building 4 [0, 0, 0, 0] 5 71
Building 4 [0, 0, 0, 0] 5 76
Building 4 [0, 0, 0, 0] 5 81
Building 4 [1, 0, 0, 0] 5 86
Building 4 [0, 0, 0, 0] 5 91
Building 4 [0, 0, 0, 0] 5 96
Building 4 [0, 0, 0, 0] 5 101
Building 4 [0, 0, 0, 0] 5 106
Building 4 [0, 0, 0, 0] 5 111
Building 4 [0, 0, 0, 0] 5 116
Building 4 [1, 0, 0, 0] 5 121
Building 4 [0, 1, 0, 0] 5 126
Building 4 [0, 0, 0, 0] 5 131
Building 4 [0, 0, 0, 0] 5 136
Building 4 [0, 0, 0, 0] 5 141
Building 4 [0, 0, 0, 0] 5 146
Building 4 [0, 0, 0, 0] 5 151
Building 4 [0, 0, 0, 0] 5 156
Building 4 [0, 0, 0, 0] 5 161
Building 4 [0, 0, 0, 0] 5 166
Building 4 [0, 0, 0, 1] 5 171
Building 4 [0, 0, 0, 0] 5 176
Building 4 [0, 0, 0, 1] 5 181
Building 4 [0, 0, 0, 0] 5 186
Building 4 [0, 0, 0, 0] 5 191
Building 4 [0, 0, 0, 0] 5 196
B